### HELLO! This is a simple tool I have created to query optical photometry for nearby galaxies at a location of you choosing from SDSS DR12 in Vizier. Hope you find it useful! --Vic 07/31/2020

In [8]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from astropy.visualization import ImageNormalize, ZScaleInterval

import IPython.display
import ipywidgets as widgets
import pickle
import dustmaps.sfd
import numpy as np
import os
from astropy import units as u
from astropy.io import fits
import astropy.wcs as wcs

# This file contains all external dependencies
from utilities import util

dustmaps.sfd.fetch()
sfd = dustmaps.sfd.SFDQuery()

Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.
Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.


# Name of GRB, metadata, coordinates, extinction

In [2]:
data_path = 'data/'
object_name = 'GRB181123B'
ra  = '12:17:27.94'
dec = '14:35:52.66'

# Parse coord
coord = util.parse_coord(ra, dec)

# Get Swift data
swift_data = util.get_grb_data(object_name)

# Get MW Av
av = sfd(coord) * 3.1
print('Milky Way Av={0} mag'.format(('%7.4f'%av).strip()))

Successfully downloaded Swift GRB data...
Got GRB metadata
Time: 05:33:03
Trigger Number: 873186
BAT T90: 0.26
XRT RA: 12:17:28.02
XRT Dec: 14:35:52.7
Milky Way Av=0.1084 mag


### Catalog search - this will grab metadata for sources around input coord

In [18]:
# This method will search and crossmatch catalogs given an input list
# of catalogs.  The output is an astropy table keyed by as catname_key
# where key is the original key in each catalog
catnames = ['strm','ps1dr2','sdssdr12','2mass','unwise','glade','des']
catalog = util.search_catalogs(coord, catnames, search_radius=2.564*u.arcmin,
    match_radius=2.0*u.arcsec, outfile=os.path.join(data_path,object_name+'.cat'))
print(catalog['ra','dec'])

Searching strm catalog...
50 records in strm catalog
50 total records in catalog
Searching ps1dr2 catalog...
1259 records in ps1dr2 catalog
1259 total records in catalog
Searching sdssdr12 catalog...
0 records in sdssdr12 catalog
Searching 2mass catalog...
0 records in 2mass catalog
Searching unwise catalog...
161 records in unwise catalog
1417 total records in catalog
Searching glade catalog...
0 records in glade catalog
Searching des catalog...
0 records in des catalog
        ra             dec     
------------------ ------------
      184.35368211  14.57843162
184.38281921499998 14.577962395
184.35767894499998  14.58613477
184.36251433500001  14.58281237
     184.364357775  14.58825537
      184.36443321  14.58973126
184.36641739499998 14.587203895
      184.36838314  14.58895907
      184.37317974 14.589222925
      184.37746584  14.58836374
               ...          ...
        184.375096   14.6260586
       184.3851607   14.6272346
       184.3662834   14.6374617
       184.3

### If you want an image of where you are looking at, you can use this!

In [19]:
# Here is an image of the cut out. I bet it looks super pretty!
# First tries SDSS, then PS1, then DES, then SkyMapper to get jpeg
# should have 100% sky coverage from those surveys
imfile = util.downloadPS1(coord, 'r')

%matplotlib widget
fig, ax = plt.subplots(figsize=(10,10))
fopen = fits.open(imfile)
image = fopen[1].data
header = fopen[1].header
wcs_object = wcs.WCS(header)
ax = plt.subplot(projection=wcs_object)
ax.imshow(image, cmap='gray', norm=ImageNormalize(image, interval=ZScaleInterval()))

plt.title(object_name,fontsize=20)
x_center, y_center = wcs_object.all_world2pix(coord.ra.deg,coord.dec.deg,1)
ax.hlines(y_center,x_center+20,x_center+300,color='limegreen',alpha=0.8,linewidth=2)
ax.vlines(x_center,y_center+20,y_center+300,color='limegreen',alpha=0.8,linewidth=2)

ax.hlines(50,170,470,color='k',linewidth=3)
ax.vlines(470,50,350,color='k',linewidth=3)
ax.text(400,400,'N',color='k',fontsize=15)
ax.text(35,50,'E',color='k',fontsize=15)

try:
    xrt_pos = util.parse_coord(swift_data['BAT RA'], swift_data['BAT Dec'][1:])
    x_xrt, y_xrt = wcs_object.all_world2pix(xrt_pos.ra.deg,xrt_pos.dec.deg,1)
    ax.add_patch(patches.Circle((x_xrt,y_xrt),radius=np.float(swift_data['BAT Error Radius'])/0.25,edgecolor='m',alpha=0.8,facecolor='none',linewidth=2,linestyle='--',label='XRT position at RA = %.3f, Dec = %.3f'%(xrt_pos.ra.deg, xrt_pos.dec.deg),picker=True))
except:
    print('No XRT position.')

for i in range(len(catalog)):
   x, y = wcs_object.all_world2pix(xrt_pos.ra.deg,xrt_pos.dec.deg,1)
   ax.add_patch(patches.Circle((x,y),radius=2,edgecolor='r',alpha=0.5,facecolor='none',linewidth=2,linestyle='-',label='Galaxy at RA = %.3f, Dec = %.3f'%(catalog['ra'][i],catalog['dec'][i]),picker=True))

ax.set_xlabel('RA')
ax.set_ylabel('Dec')
fig.canvas.mpl_connect('click_pos', lambda event: onclick())
fig.canvas.mpl_connect('pick_event', lambda event: onpick())

We need to download 1 PS1 images.
Wrote out: rings.v3.skycell.1635.064.stk.r.unconv_mod.fits


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

11

### Once you have the results, there are several things you can do. One of them is finding the probability chance coincident (Pcc) of the nearby galaxies which essentially tells you how likely that the GRB originated from a galaxy. The lower the Pcc, the more probable. 

In [ ]:
# The function you can use to calcilate that is here:
def pcc(r,m): 
    sigma = (1/(0.33*np.log(10)))*10**(0.33*(m-24)-2.44) 
    prob = 1-np.exp(-(np.pi*(r**2)*sigma)) 
    return prob

# An example of what a plot of Pcc as a function of distance looks like is here. This is what I did for GRB200522A. 
import numpy as np
with open(object_name+'.txt', 'rb') as fp:
    result = pickle.load(fp)

# Make a list of r_mag that corresponds to each galaxy near the GRB
r_mag = [line[7] for line in result]
#print(r_mag)

# Distance is in arcsecond, not arcminute of the galaxy from the GRB
distance = [line[0]*60 for line in result]
#print(distance)

# Make a list of photometric redshift
phot = [line[10] for line in result]
#print(phot)
    
prob = []
for i in range(len(r_mag)):
    prob.append(pcc(distance[i],r_mag[i]))
        
#print(prob)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]
# = plt.subplot(21.137, 0.316, c=0.5537)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.linewidth'] = 2
plt.scatter(distance, prob, marker = "o", c=phot, cmap='plasma', alpha = 0.6, s=195, edgecolors='black')
plt.ylabel('P$_{cc}$', fontsize = 20)
plt.xlabel('Distance (arcsec)', fontsize = 20)
clb = plt.colorbar()
clb.solids.set_alpha(1)
clb.set_label('phot-z', fontsize = 20, labelpad=15, rotation=90)
plt.savefig(object_name+'_pcc.png', format = 'pdf', dpi=300, bbox_inches="tight")
plt.show()

#### You can also sort out other lists of information such as the redshift and distance to make a scatter plot of redshifts as a function of distance. That can be helpful for figuring out whether the host is in a galaxy cluster or not. 